# Word Search II

Given an `m x n` board of characters and a list of strings `words`, return all words on the board.

Each word must be constructed from letters of sequentially adjacent cells, where adjacent cells are horizontally or vertically neighboring. The same letter cell may not be used more than once in a word.

## Examples

**Example 1:**
```
Input: board = [["o","a","a","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]], words = ["oath","pea","eat","rain"]
Output: ["eat","oath"]
```

**Example 2:**
```
Input: board = [["a","b"],["c","d"]], words = ["abcb"]
Output: []
```

In [3]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False


class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True


class Solution:
    def findWords(self, board: list[list[str]], words: list[str]) -> list[str]:
        # Build the Trie
        trie = Trie()
        for word in words:
            trie.insert(word)

        m, n = len(board), len(board[0])
        result = set()

        def backtrack(i, j, node, path):
            char = board[i][j]
            if char not in node.children:
                return

            # Move to the next Trie node
            node = node.children[char]
            path.append(char)

            # If it's the end of a word, add it to the result
            if node.is_end_of_word:
                result.add("".join(path))

            # Mark the cell as visited
            board[i][j] = "#"

            # Explore neighbors
            for x, y in [(i + 1, j), (i - 1, j), (i, j + 1), (i, j - 1)]:
                if 0 <= x < m and 0 <= y < n and board[x][y] != "#":
                    backtrack(x, y, node, path)

            # Restore the cell and path
            board[i][j] = char
            path.pop()

        # Start backtracking from each cell
        for i in range(m):
            for j in range(n):
                backtrack(i, j, trie.root, [])

        return list(result)

## Optimization Ideas for the Code

### 1. Dynamic Trie Pruning
- **Current Behavior**:
  - When a word is found, its end marker (`"#"`) is removed from the Trie.
  - Additionally, if a Trie node becomes empty after backtracking, it is deleted using `parent.pop(char)`.
- **Why It's Efficient**:
  - Dynamically reduces the search space of the Trie during execution.
  - Prevents revisiting already found words, saving unnecessary computations.

### 2. Avoid Redundant String Operations
- **Current Behavior**:
  - `path.append(char)` and `"".join(path)` are used to construct the matched word.
- **Optimization Idea**:
  - Instead of appending characters to a list and joining them later, track the indices of the current word in the grid. This avoids creating intermediate strings during recursion.

### 3. Mark Visited Cells Efficiently
- **Current Behavior**:
  - The visited cell is marked by replacing the character in `board[i][j]` with `"#"`, and it is restored after backtracking.
- **Optimization Idea**:
  - Use a separate `visited` set or a bitmask to track visited cells, avoiding in-place modifications to `board`. This can improve memory locality and avoid potential side effects.

### 4. Heuristic Sorting
- **Optimization Idea**:
  - Sort words or Trie nodes by their frequency or length before the search starts.
  - This can prioritize matching shorter or more frequent words, which leads to faster Trie pruning and early termination.

### 5. Reduce Neighbor Search Overhead
- **Current Behavior**:
  - For every cell, all four neighbors are checked using `0 <= x < m and 0 <= y < n`.
- **Optimization Idea**:
  - Precompute valid neighbor coordinates for the grid or avoid unnecessary boundary checks with better condition handling. This reduces redundant comparisons.

### 6. Batch Word Search for Large Grids
- **Optimization Idea**:
  - For very large grids, segment the search space into smaller regions, processing parts of the board incrementally.
  - This helps minimize Trie depth for each segment and reduces memory overhead.


## Detailed Explanation of Current Code’s Efficiency

### 1. Trie Construction
- Efficiently stores all words in a prefix tree, which allows quick lookup for valid prefixes and words.
- **Space Complexity**: \(O(W \cdot L)\), where \(W\) is the number of words, and \(L\) is the average word length.

### 2. Backtracking with Trie
- Each cell in the grid is a starting point for recursive exploration.
- By only exploring valid paths in the Trie (`if board[x][y] in node`), unnecessary branches are pruned early.

### 3. Dynamic Word Removal
- Found words are dynamically removed from the Trie (`node.pop("#")`), which significantly reduces the search space during subsequent searches.

### 4. In-Place Board Modification
- Marking cells with `"#"` avoids using an additional data structure to track visited cells.
- This keeps space usage minimal, though it modifies the input grid.

$\quad$ The above code can be optimized as follows:

In [4]:
class Solution:
    def findWords(self, board: list[list[str]], words: list[str]) -> list[str]:
        # Build Trie
        trie = {}
        for word in words:
            node = trie
            for char in word:
                if char not in node:
                    node[char] = {}
                node = node[char]
            node["#"] = True  # Mark the end of a word

        m, n = len(board), len(board[0])
        result = []

        def backtrack(i, j, parent, path):
            char = board[i][j]
            node = parent[char]
            path.append(char)

            # If we find a word, add to result and remove it from the Trie
            if "#" in node:
                result.append("".join(path))
                # Remove the end marker to avoid duplicate matching
                node.pop("#")

            # Mark the cell as visited
            board[i][j] = "#"

            # Explore neighbors
            for x, y in [(i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)]:
                if 0 <= x < m and 0 <= y < n and board[x][y] in node:
                    backtrack(x, y, node, path)

            # Restore the cell
            board[i][j] = char
            path.pop()

            # Prune the Trie node if it's empty
            if not node:
                parent.pop(char)

        # Start backtracking
        for i in range(m):
            for j in range(n):
                if board[i][j] in trie:
                    backtrack(i, j, trie, [])

        return result